In [1]:
import sagemaker
import boto3
import warnings
from time import gmtime, strftime

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Initialize SageMaker session and client
sagemaker_session = sagemaker.Session()
sm_client = boto3.client('sagemaker')
role = sagemaker.get_execution_role()  # Replace with your SageMaker execution role
warnings.filterwarnings("ignore")

In [3]:
sagemaker_bucket_name = "hemz-sagemaker-bucket"

inference_image_uri = "654654222480.dkr.ecr.ap-south-1.amazonaws.com/prophet_infernece:latest"

model_name = "Prophet-serverless-model" + "-" + strftime("%Y%m%d%H%M%S", gmtime())

endpoint_config_name = "prophet-endpoint-config"

endpoint_name = "prophet-endpoint"

In [4]:
# List training jobs
response = sm_client.list_training_jobs(
    SortBy='CreationTime',
    SortOrder='Descending'
)

# Print training jobs to find the relevant one
latest_training_job_name = response['TrainingJobSummaries'][0]['TrainingJobName'] \
    if response['TrainingJobSummaries'][0]['TrainingJobStatus'] == 'Completed' \
    else print("The latest training Job is not successful")

In [5]:
# Create a model for endpoint
sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    Containers=[{
        "Image": inference_image_uri,
        "Mode": "SingleModel",
        "ModelDataUrl": f's3://{sagemaker_bucket_name}/output-data/model-artifacts/{latest_training_job_name}/output/model.tar.gz'
    }]
)

{'ModelArn': 'arn:aws:sagemaker:ap-south-1:654654222480:model/Prophet-serverless-model-20240818124810',
 'ResponseMetadata': {'RequestId': '8f166f1c-605b-47ed-83ab-dde9faaf4eca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8f166f1c-605b-47ed-83ab-dde9faaf4eca',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '102',
   'date': 'Sun, 18 Aug 2024 12:48:10 GMT'},
  'RetryAttempts': 0}}

In [6]:
# Create endpoint configuration
endpoint_config = sm_client.create_endpoint_config(
   EndpointConfigName="prophet-endpoint-config",
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "AllTraffic",
            "ServerlessConfig": {
                "MemorySizeInMB": 2048,
                "MaxConcurrency": 1,
                "ProvisionedConcurrency": 1,
            }
        } 
    ]
)

In [7]:
# Create the endpoint
sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

{'EndpointArn': 'arn:aws:sagemaker:ap-south-1:654654222480:endpoint/prophet-endpoint',
 'ResponseMetadata': {'RequestId': '773a62ab-8654-45ca-8eed-02c145200ef8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '773a62ab-8654-45ca-8eed-02c145200ef8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '85',
   'date': 'Sun, 18 Aug 2024 12:48:10 GMT'},
  'RetryAttempts': 0}}